In [2]:
# ================================================================
# Lab-04  Secure Multi-Party Computation  (Seiyun University 2025)
# ================================================================
!pip install -q sympy
import random, sympy, json, base64
from typing import List, Tuple

# ----------------------------------------------------------
# PART-I  MPC Sum via Additive Secret Sharing (3 parties)
# ----------------------------------------------------------
def share(value: int, modulus: int = 2**61-1) -> Tuple[int,int,int]:
    """Split value into 3 additive shares mod p."""
    p = modulus
    r1 = random.randint(0, p-1)
    r2 = random.randint(0, p-1)
    r3 = (value - r1 - r2) % p
    return r1, r2, r3

def reconstruct(*shares: int, modulus: int = 2**61-1) -> int:
    return sum(shares) % modulus

# Private inputs
salary_Ahmed = 70
salary_Ali   = 80
salary_Rash  = 70
p = 2**61-1

# Step-1: each party creates shares
shA1, shA2, shA3 = share(salary_Ahmed, p)  # Ahmed keeps shA1, sends shA2 to Ali, shA3 to Rash
shB1, shB2, shB3 = share(salary_Ali,   p)
shC1, shC2, shC3 = share(salary_Rash,  p)

# Step-2: local sum of received shares
local_Ahmed = (shA1 + shB1 + shC1) % p
local_Ali   = (shA2 + shB2 + shC2) % p
local_Rash  = (shA3 + shB3 + shC3) % p

# Step-3: reconstruct global sum
global_sum = reconstruct(local_Ahmed, local_Ali, local_Rash, p)
print("=== MPC Sum ===")
print("Private inputs:", salary_Ahmed, salary_Ali, salary_Rash)
print("Reconstructed sum:", global_sum)

# ----------------------------------------------------------
# PART-II  Shamir (k,n) Secret Sharing
# ----------------------------------------------------------
PRIME = 2**127-1   # 127-bit Mersenne prime
def eval_poly(coeffs: List[int], x: int, p: int = PRIME) -> int:
    """Evaluate polynomial with Horner rule mod p."""
    res = 0
    for c in reversed(coeffs):
        res = (res * x + c) % p
    return res

def shamir_share(secret: int, k: int, n: int, p: int = PRIME) -> List[Tuple[int,int]]:
    """Generate n shares for (k,n) scheme."""
    coeffs = [secret] + [random.randint(1, p-1) for _ in range(k-1)]
    shares = [(i, eval_poly(coeffs, i, p)) for i in range(1, n+1)]
    return shares

def shamir_reconstruct(shares: List[Tuple[int,int]], p: int = PRIME) -> int:
    """Lagrange interpolation mod p."""
    k = len(shares)
    xs, ys = zip(*shares)
    secret = 0
    for i in range(k):
        num, den = 1, 1
        for j in range(k):
            if i != j:
                num = num * (-xs[j]) % p
                den = den * (xs[i] - xs[j]) % p
        inv_den = pow(den, p-2, p)
        lagrange = num * inv_den % p
        secret = (secret + ys[i] * lagrange) % p
    return secret

# Demo: (3,5) scheme
secret = 123456789
shares = shamir_share(secret, k=3, n=5)
print("\n=== Shamir (3,5) ===")
print("Shares:", shares)
# any 3 shares
recovered = shamir_reconstruct([shares[0], shares[2], shares[4]])
print("Recovered secret:", recovered)

# ----------------------------------------------------------
# PART-III  Tiny Garbled AND Gate (Omar has A, Nancy has B)
# ----------------------------------------------------------
WIRE_ZERO = 0xAA, 0xBB   # garbled labels for 0
WIRE_ONE  = 0xCC, 0xDD   # garbled labels for 1

GARBLED_TABLE = [
    (0xAA ^ 0xBB, 0x00),   # 0 AND 0 -> 0
    (0xAA ^ 0xDD, 0x00),   # 0 AND 1 -> 0
    (0xCC ^ 0xBB, 0x00),   # 1 AND 0 -> 0
    (0xCC ^ 0xDD, 0x01)    # 1 AND 1 -> 1
]

def garbled_and(a: int, b: int) -> int:
    """a,b are bits; returns a AND b without revealing inputs."""
    # In real OT the evaluator receives the correct label; here we simulate.
    key = (WIRE_ZERO[a] ^ WIRE_ZERO[b]) if (a==0 and b==0) else \
          (WIRE_ZERO[a] ^ WIRE_ONE [b]) if (a==0 and b==1) else \
          (WIRE_ONE [a] ^ WIRE_ZERO[b]) if (a==1 and b==0) else \
          (WIRE_ONE [a] ^ WIRE_ONE [b])
    for k, out in GARBLED_TABLE:
        if k == key:
            return out
    raise RuntimeError("Garbled key not found")

print("\n=== Garbled AND ===")
A, B = 1, 0
print(f"Omar input A={A}, Nancy input B={B}")
print(f"Garbled output A AND B =", garbled_and(A,B))

=== MPC Sum ===
Private inputs: 70 80 70
Reconstructed sum: 220

=== Shamir (3,5) ===
Shares: [(1, 78014418889060137046315727115187459076), (2, 45065595704495286429553563756216752533), (3, 71294713906774679881400813639095442887), (4, 156701773495898317401857476763823530138), (5, 131145591011396967259236249414516908559)]
Recovered secret: 123456789

=== Garbled AND ===
Omar input A=1, Nancy input B=0
Garbled output A AND B = 0
